In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/02/28 09:04:53] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/28 09:04:53] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/28 09:04:53] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [4]:
gui = GUI('data/input/2.jpg')
gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
# gui.load_detection_result()
gui.show_detection_result()

[2022/02/28 09:04:56] root DEBUG: dt_boxes num : 30, elapse : 0.19381284713745117
[2022/02/28 09:04:56] root DEBUG: cls num  : 30, elapse : 0.10383129119873047
[2022/02/28 09:04:56] root DEBUG: rec_res num  : 30, elapse : 0.14563560485839844
[Text Detection Completed in 0.498 s] Input: data/input/2.jpg Output: data/output/ocr/2.json
[Compo Detection Completed in 0.370 s] Input: data/input/2.jpg Output: data/output/ip/2.json
[Merge Completed] Input: data/input/2.jpg Output: data/output/merge/2.jpg


In [5]:
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.shrink_target_element_img()
    op.resize(gui.detection_resize_width, gui.detection_resize_height)        
    op.detect_text(paddle_ocr)
    operations.append(op)
#     op.show_target_ele()

[2022/02/28 09:05:00] root DEBUG: dt_boxes num : 0, elapse : 0.002778768539428711
[2022/02/28 09:05:00] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 09:05:00] root DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[Text Detection Completed in 0.006 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 09:05:02] root DEBUG: dt_boxes num : 3, elapse : 0.013417720794677734
[2022/02/28 09:05:02] root DEBUG: cls num  : 3, elapse : 0.01670551300048828
[2022/02/28 09:05:02] root DEBUG: rec_res num  : 3, elapse : 0.04688096046447754
[Text Detection Completed in 0.081 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 09:05:02] root DEBUG: dt_boxes num : 0, elapse : 0.0034732818603515625
[2022/02/28 09:05:02] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 09:05:02] root DEBUG: rec_res num  : 0, elapse : 9.5367431640625e-07
[Text Detection Completed in 0.006 s] Input: data/operation/target.png Output: data/operation/target.

[Text Detection Completed in 0.015 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 09:05:13] root DEBUG: dt_boxes num : 0, elapse : 0.0027914047241210938
[2022/02/28 09:05:13] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 09:05:13] root DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[Text Detection Completed in 0.005 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 09:05:14] root DEBUG: dt_boxes num : 3, elapse : 0.014405965805053711
[2022/02/28 09:05:14] root DEBUG: cls num  : 3, elapse : 0.015900373458862305
[2022/02/28 09:05:14] root DEBUG: rec_res num  : 3, elapse : 0.030788898468017578
[Text Detection Completed in 0.065 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 09:05:14] root DEBUG: dt_boxes num : 0, elapse : 0.00454401969909668
[2022/02/28 09:05:14] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 09:05:14] root DEBUG: rec_res num  : 0, elapse : 1.430511474609

In [10]:
opr = operations[1]
gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model)

In [9]:
# ***************************
# *** Streaming Detection ***
# ***************************

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite('data/output/temp.jpg', frame)
        gui = GUI('data/output/temp.jpg')
        gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
        cv2.imshow('result', gui.det_result_imgs['merge'])
        print('===================\n')
cap.release()
cv2.destroyAllWindows()

[2022/02/26 13:25:22] root DEBUG: dt_boxes num : 0, elapse : 0.14470863342285156
[2022/02/26 13:25:22] root DEBUG: cls num  : 0, elapse : 0
[2022/02/26 13:25:22] root DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[Text Detection Completed in 0.161 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 1.063 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

